# Importation des bibliothèques 

In [1]:
# Importations de base des bibliothèques de manipulation de données
import pandas as pd
import numpy as np
import ast

# Importations pour la sérialisation et le traitement des données
import pickle

# Importations pour la modélisation et le traitement du texte
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics.pairwise import cosine_similarity
import gensim

# Importations des fonctions personnalisées
import init as func

# Configuration pour filtrer les avertissements de sklearn
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bille\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\bille\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\bille\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Charger les données nettoyées:

#### Chargement Dataframe:

In [ ]:
df = pd.read_csv('Data/stack_overflow_data_cleaned.csv')
display(df.head())
print(df.shape)

#### Les valeurs maqnuantes pour df:

In [ ]:
func.taux_de_Remplissage_tableau(df, affichage_all = True)

#### Chargement du dataframe Train_df:

In [ ]:
train_df = pd.read_csv("Data/stack_overflow_data_cleaned_train.csv")
display(train_df.head())
print(train_df.shape)

#### Les valeurs manquantes pour train_df:

In [ ]:
func.taux_de_Remplissage_tableau(train_df, affichage_all = True)

#### Chargement du dataframe Test_df:

In [ ]:
test_df = pd.read_csv("Data/stack_overflow_data_cleaned_test.csv")
display(test_df.head())
print(test_df.shape)

#### Les valeurs manquantes pour test_df:

In [ ]:
func.taux_de_Remplissage_tableau(test_df, affichage_all = True)

#### Suppression des valeurs manquantes sur la colonne cleaned_title

In [ ]:
df = df.dropna()
train_df = train_df.dropna()
test_df = test_df.dropna()

In [ ]:
print("La taille du dataset df après suppressions des valeurs manquantes : ", df.shape)
print("La taille du dataset train_df après suppressions des valeurs manquantes : ", train_df.shape)
print("La taille du dataset test_df après suppressions des valeurs manquantes : ", test_df.shape)

#### Création de la liste du vocabulaire des mots les plus fréquents de corpus df['combined_title_body']:

In [ ]:
# Combiner tout le contenu des corps de texte nettoyés en une seule chaîne de texte
corpus_combined_title_body = " ".join(df['combined_title_body'].values).lower()

# Afficher la fréquence de chaque mot dans le corpus des corps de texte nettoyés
corpus_combined_title_body_tokens = corpus_combined_title_body.split()
value_counts_combined_title_body = pd.Series(corpus_combined_title_body_tokens).value_counts()
print("Fréquence de chaque mot dans le corpus des corps de texte nettoyés :\n", value_counts_combined_title_body)

À partir des résultats de value_counts obtenus pour le corpus df['combined_title_body'], je sélectionnerai les 200 mots les plus fréquents. Ces mots seront ensuite intégrés dans la liste vocabulary: 

In [ ]:
# Créer la liste du vocabulaire des mots les plus fréquents du corpus
vocabulary = list(value_counts_combined_title_body.head(200).index)
print("Les 200 mots les plus fréquents du corpus des corps de texte nettoyés :\n", vocabulary)

#### Création des vectoriseurs

Ensuite, ce vocabulary sera intégré dans le TfidfVectorizer que j'utiliserai pour transformer le texte en vecteurs numériques:

In [ ]:
vectorizer = TfidfVectorizer(vocabulary=vocabulary)
# vectorizer = TfidfVectorizer(max_features=5000)
# vectorizer = count_vectorizer(max_features=5000)  # Vectorisation vectorizer avec un maximum de 5000 caractéristiques

**Remarque :** Pour des tâches de modélisation de sujets (LDA, NMF) ou de classification de texte, il est généralement recommandé d'utiliser `TfidfVectorizer`. Ce dernier prend en compte l'importance relative des termes en calculant le produit de la fréquence des termes (TF) et de la fréquence inverse des documents (IDF). Cette approche réduit l'importance des mots très fréquents dans le corpus et augmente celle des mots plus rares mais informatifs. À l'inverse, `CountVectorizer` compte uniquement le nombre d'occurrences de chaque mot dans les documents. Il peut être utile si la simplicité et la rapidité sont des priorités, ou si vos données sont petites et peu sujettes aux biais introduits par les mots fréquents.

#### Vectoriser les textes des questions avec vectorizer

Comme je l'ai mentionné dans le notebook exploratoire, je vais faire le fit le TfidfVectorizer sur la colonne `train_df['cleaned_title']` des données d'entraînement. Ensuite, je l'utiliserai pour transformer les données des colonnes `train_df['combined_title_body']` pour l'ensemble d'entraînement et `test_df['combined_title_body']` pour l'ensemble de test:

In [ ]:
vectorizer.fit(train_df['cleaned_title'])  # Fit sur les titres des données d'entraînement
bow_train_combined = func.vectorizer_transform(train_df['combined_title_body'],vectorizer) # Transformation des titres + corps des données d'entraînement
bow_test_combined = func.vectorizer_transform(test_df['combined_title_body'],vectorizer )  # Transformation des titres + corps des données de test

#### Enregistrer le CountVectorizer au format .pkl:

Je vais enregistrer le TfidfVectorizer au format .pkl afin de l'utiliser ultérieurement dans l'API que je suis en train de construire:

In [ ]:
with open('Model/unsupervised/vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

#### Détermination du nombre optimal de topics:

#### Préparer les données pour Gensim

Je prépare les données (paramètres) à fournir à Gensim pour calculer la cohérence et la perplexité. Voici les étapes :

1. **Tokenisation** : Les documents sont convertis en listes de mots pour faciliter les analyses futures. La taille du dataset tokenisé et des exemples sont également présentés pour visualiser cette étape.

2. **Dictionnaire** : À partir des mots tokenisés, je crée un dictionnaire avec Gensim. Cette étape permet une référence rapide et efficace, illustrée par des exemples et la quantité de mots uniques.

3. **Bag-of-Words** : Je transforme les listes de mots en tuples qui indiquent la fréquence des mots, facilitant ainsi l'analyse quantitative du texte.

In [ ]:
# Séparer chaque document dans la colonne 'cleaned_body' en une liste de mots
texts = [doc.split() for doc in train_df['combined_title_body']]  # Tokenisation des textes
print("Séparation des documents en listes de mots (exemple):", texts[0])
print("La taille de 'texts':", len(texts))
print('\n')

# Créer un dictionnaire à partir des textes, chaque mot unique dans les textes est une clé dans le dictionnaire
dictionary = gensim.corpora.Dictionary(texts)  # Création du dictionnaire
print("Dictionnaire créé à partir des textes (exemple):", dict(list(dictionary.token2id.items())[:10]))
print("La taille du dictionnaire:", len(dictionary))
print('\n')

# Convertir les textes en une liste de sacs de mots (Bag-of-Words), chaque document est représenté par un ensemble de tuples (id_mot, fréquence)
corpus = [dictionary.doc2bow(text) for text in texts]  # Création du corpus
print("Conversion des textes en sacs de mots (Bag-of-Words) (exemple):", corpus[:2])
print("La taille du corpus:", len(corpus))

#### Calculer les valeurs de cohérence et perplexité pour différents nombres de topics

Avec les éléments tels que le dictionary, corpus, texts, et bow_train_combined, nous extrayons les listes suivantes : model_list, coherence_values et perplexity_values:

In [ ]:
# Calcul des valeurs de cohérence et de perplexité
model_list, coherence_values, perplexity_values = func.compute_coherence_perplexity(dictionary, corpus, texts, bow_train_combined, limit=40, start=2, step=1)

#### Afficher le coherence score et la perplexité pour chaque modèle

Grâce à ces listes, j'affiche le graphique de l'évolution de la cohérence et de la perplexité en fonction du nombre de topics:

In [ ]:
func.plot_coherence_and_perplexity(coherence_values, perplexity_values)

#### Sélectionner le nombre optimal de topics 

In [ ]:
# Sélection du nombre optimal de topics
optimal_index = coherence_values.index(max(coherence_values))
optimal_num_topics = range(2, 40)[optimal_index]
optimal_coherence = max(coherence_values)
optimal_perplexity = perplexity_values[optimal_index]

# Affichage des résultats
print(f'Nombre optimal de topics: {optimal_num_topics}')
print(f'Valeur maximale de cohérence: {round(optimal_coherence,2)}')
print(f'Perplexité au nombre optimal de topics: {round(optimal_perplexity,2)}')

**Conclusion :**  
L'analyse du graphique démontre que le nombre optimal de topics pour le modèle est de 16, où le score de cohérence atteint son maximum de 0.5, indiquant une très bonne cohérence thématique. Cette valeur maximale de cohérence suggère que le modèle avec 16 topics est particulièrement efficace pour segmenter le corpus en thèmes pertinents et bien définis. De plus, à ce même niveau, la perplexité est de 450.94, ce qui est relativement élevé, indiquant que le modèle pourrait être complexe mais bien généralisé. Cela confirme que 16 topics offrent une balance optimale entre la clarté des thèmes extraits et la capacité du modèle à expliquer les données de manière cohérente et compréhensible.

#### Création et entraînement du modèle LDA 

On entraîne le modèle LDA avec le nombre optimal de topics 16:

In [ ]:
# Création et entraînement des modèles LDA et NMF
lda_model = LatentDirichletAllocation(n_components=optimal_num_topics, random_state=42)  # Modèle LDA avec le nombre optimal de topics

Avec ce modèle LDA (lda_model), on applique ensuite un fit_transform sur le bag of words des données d'entraînement de la colonne `train_df['combined_title_body']` :

In [ ]:
lda_output  = func.prediction(bow_train_combined, lda_model)

#### Fonction pour Visualiser les mots les plus importants pour chaque topic:

#### Visualisation des topics pour LDA:

Je vais visualiser les mots les plus pertinents de chaque topic généré par notre modèle LDA. J'ai défini n_top_words à 10, ce qui signifie que je vais afficher les dix mots les plus significatifs pour chaque topic identifié:

In [ ]:
n_top_words = 10  # Nombre de mots à afficher par topic
feature_names = vectorizer.get_feature_names_out()  # Noms des caractéristiques
func.plot_top_words(lda_model , feature_names, n_top_words, 'Topics in LDA model', optimal_num_topics)

**Conclusion:** 

La visualisation des mots clés pour chaque topic, tels que "php", "sql", "android", et "java", clarifie les thèmes technologiques dominants et améliore la précision des prédictions du modèle LDA. Ces mots clés permettent au modèle de catégoriser plus efficacement les nouveaux documents, rendant les prédictions plus pertinentes et ciblées.

#### Enregistrer le modèle LDA au format .pkl

In [ ]:
with open('Model/unsupervised/lda_model.pkl', 'wb') as file:
    pickle.dump(lda_model, file)

#### Visualiser les topics en 2D avec t-SNE

In [ ]:
func.plot_tsne_lda(lda_output)

**Conclusion :**

La visualisation en t-SNE des résultats LDA révèle une distribution claire des topics dans un espace 2D, avec des clusters distincts qui montrent des regroupements de sujets similaires et des transitions qui suggèrent des liens thématiques entre certains d'entre eux. Cette approche offre des insights sur la séparation et les interactions entre les topics, facilitant l'ajustement du modèle et l'analyse détaillée du corpus.

### Création des matrices de probabilités Mtopics-words

Ensuite, je vais extraire et normaliser les probabilités des mots dans chaque topic (la matrice des topics words) du modèle LDA à partir de l'attribut components_:

In [ ]:
Mtopics_words_lda = lda_model.components_   # Normalisation des composantes

#### La Matrice M_train_quest-topics lda:

Je vais récupérer la matrice des topics des questions d'entraînement déterminés par LDA (obtenue en appliquant un fit_transform avec le modèle LDA sur les données d'entraînement de la colonne `train_df['combined_title_body']`), stockée dans `lda_output`, et l'assigner à la variable `M_train_quest_topics_lda`:

In [ ]:
M_train_quest_topics_lda = lda_output  # Sortie LDA pour les données d'entraînement

#### La Matrice M_test_quest-topics lda:

Je récupère aussi la matrice des topics des questions de test déterminés par LDA (obtenue en appliquant une prédiction avec le modèle LDA sur le bag-of-words des données de test de la colonne `test_df['combined_title_body']` ), et l'assigner à la variable `M_test_quest_topics_lda`:

In [ ]:
M_test_quest_topics_lda = func.prediction(bow_test_combined, lda_model, fit_transform=False)
M_test_quest_topics_lda.shape

#### Enregistrer la matrice dans un fichier .npz

In [ ]:
with open('Model/unsupervised/train_topics_lda.pkl', 'wb') as file:
    pickle.dump(M_train_quest_topics_lda, file)

#### Calcul des probabilités de mots pour les questions Train et Test avec application d'un filtre:

On prend la matrice des topics des questions d'entraînement et le modèle LDA, puis on effectue **un produit matriciel** pour produire une matrice des mots spécifiques à chaque topic. Le résultat, assigné à `M_train_quest_words_lda`, représente ainsi les mots clés filtrés en fonction de leur pertinence pour chaque topic identifié:

In [ ]:
M_train_quest_words_lda = func.calculate_words(M_train_quest_topics_lda, lda_model)
M_train_quest_words_lda.shape

On prend aussi la matrice des topics des questions de test et le modèle LDA, puis on effectue **un produit matriciel** pour produire une matrice des mots spécifiques à chaque topic. Le résultat, assigné à `M_test_quest_words_lda`, représente ainsi les mots clés filtrés en fonction de leur pertinence pour chaque topic identifié :

In [ ]:
M_test_quest_words_lda = func.calculate_words(M_test_quest_topics_lda, lda_model)
M_test_quest_words_lda.shape

##### Application d'un seuil pour ne garder que les principaux mots:

On applique sur la matrice `M_test_quest_words_lda` un seuil de `threshold=0.0001` pour mettre à 0 les fréquences des mots inférieures à `threshold`, ce qui permet de garder uniquement les mots avec les plus grandes probabilités. Le résultat, assigné à `M_test_quest_words_filtered_lda`, est obtenu via la fonction `filter_words` :

In [ ]:
M_test_quest_words_filtered_lda = func.filter_words(M_test_quest_words_lda, threshold=0.0001)
M_test_quest_words_filtered_lda.shape

#### Création de DataFrames avec les mots clés proposés

Ensuite, je construis le DataFrame `df_keywords_test_lda` en utilisant les noms des mots extraits par le vectoriseur et les mots filtrés de la matrice `M_test_quest_words_filtered_lda`. Les noms des colonnes du DataFrame sont obtenus via `vectorizer.get_feature_names_out()` et les index sont alignés avec ceux de `test_df`. Le résultat est le DataFrame `df_keywords_test_lda`, qui contient les mots clés pertinents pour chaque document de test :

In [ ]:
words = vectorizer.get_feature_names_out()
df_keywords_test_lda = pd.DataFrame(M_test_quest_words_filtered_lda, columns=words, index=test_df.index) 
df_keywords_test_lda

On va utiliser ce DataFrame `df_keywords_test_lda` pour créer une colonne `predicted_lda` dans le DataFrame `test_df` :

In [ ]:
test_df.head()

In [ ]:
test_df['predicted_lda'] = df_keywords_test_lda.apply(lambda row: row.nlargest(len(test_df.split_tags)).index.tolist(), axis=1)

In [ ]:
test_df.head()

## Tester le model NMF:

On entraîne le modèle NMF avec le nombre optimal de topics, qui est de 16 :

In [ ]:
nmf_model = NMF(n_components=optimal_num_topics, random_state=42)  # Modèle NMF avec le nombre optimal de topics

Avec ce modèle NMF (`nmf_model`), on applique ensuite un `fit_transform` sur le bag of words des données d'entraînement de la colonne `train_df['combined_title_body']` :

In [ ]:
nmf_output = nmf_model.fit_transform(bow_train_combined)  # Ajustement et transformation des données d'entraînement

In [ ]:
with open('Model/unsupervised/nmf_model.pkl', 'wb') as file:
    pickle.dump(nmf_model, file)

#### Visualisation des topics pour NMF

Je vais visualiser les mots les plus pertinents de chaque topic généré par notre modèle NMF. J'ai défini `n_top_words` à 10, ce qui signifie que je vais afficher les dix mots les plus significatifs pour chaque topic identifié :

In [ ]:
func.plot_top_words(nmf_model, vectorizer.get_feature_names_out(), n_top_words, 'Topics in NMF model', optimal_num_topics)

**Conclusion :**

La visualisation des mots clés pour chaque topic, tels que "use", "file", "code", "error", et "thanks", clarifie les thèmes dominants identifiés par le modèle NMF. Ces mots clés permettent au modèle de catégoriser plus efficacement les nouveaux documents, rendant les prédictions plus pertinentes et ciblées. Les visualisations montrent que le modèle est capable d'extraire des mots clés significatifs et de les associer à des topics spécifiques, ce qui améliore la compréhension et l'interprétation des thèmes sous-jacents dans le corpus de données.

#### Création des matrices de probabilités

Ensuite, je vais extraire et normaliser les probabilités des mots dans chaque topic (la matrice des topics words) du modèle NMF à partir de l'attribut `components_` :

In [ ]:
Mtopics_words_nmf = nmf_model.components_ 

#### La Matrice M_train_quest-topics NMF:

Je vais récupérer la matrice des topics des questions d'entraînement déterminés par NMF (obtenue en appliquant un `fit_transform` avec le modèle NMF sur les données d'entraînement de la colonne `train_df['combined_title_body']`), stockée dans `nmf_output`, et l'assigner à la variable `M_train_quest_topics_nmf` :

In [ ]:
M_train_quest_topics_nmf = nmf_output  # Sortie NMF pour les données d'entraînement
M_train_quest_topics_nmf.shape

#### La Matrice M_test_quest-topics NMF:

Je récupère aussi la matrice des topics des questions de test déterminés par NMF (obtenue en appliquant une prédiction avec le modèle NMF sur le bag-of-words des données de test de la colonne `test_df['combined_title_body']`), et l'assigner à la variable `M_test_quest_topics_nmf` :

In [ ]:
M_test_quest_topics_nmf = func.prediction(bow_test_combined, nmf_model) # Transformation des données de test
M_test_quest_topics_nmf.shape

In [ ]:
with open('Model/unsupervised/train_topics_nmf.pkl', 'wb') as file:
    pickle.dump(M_train_quest_topics_nmf, file)

#### Calcul des probabilités de mots pour les questions Train et Test

On prend la matrice des topics des questions d'entraînement et le modèle NMF, puis on effectue **un produit matriciel** pour produire une matrice des mots spécifiques à chaque topic. Le résultat, assigné à `M_train_quest_words_nmf`, représente ainsi les mots clés filtrés en fonction de leur pertinence pour chaque topic identifié :

In [ ]:
M_train_quest_words_nmf = func.calculate_words(M_train_quest_topics_nmf, nmf_model)  # Multiplication matricielle pour les données d'entraînement
M_train_quest_words_nmf.shape

On prend aussi la matrice des topics des questions de test et le modèle NMF, puis on effectue **un produit matriciel** pour produire une matrice des mots spécifiques à chaque topic. Le résultat, assigné à `M_test_quest_words_nmf`, représente ainsi les mots clés filtrés en fonction de leur pertinence pour chaque topic identifié :

In [ ]:
M_test_quest_words_nmf = func.calculate_words(M_test_quest_topics_nmf, nmf_model) # Multiplication matricielle pour les données de test
M_test_quest_words_nmf.shape

# Application d'un seuil pour ne garder que les principaux mots

On applique sur la matrice `M_test_quest_words_nmf` un seuil de `threshold=0.00001` pour mettre à 0 les fréquences des mots inférieures à ce seuil, ce qui permet de garder uniquement les mots avec les plus grandes probabilités. Le résultat, assigné à `M_test_quest_words_filtered_nmf`, est obtenu via la fonction `filter_words` :

In [ ]:
M_test_quest_words_filtered_nmf = func.filter_words(M_test_quest_words_nmf, threshold=0.00001)  # Filtrage des mots basés sur le seuil
M_test_quest_words_filtered_nmf.shape

#### Création de DataFrames avec les mots clés proposés

Ensuite, je construis le DataFrame `df_keywords_test_nmf` en utilisant les noms des mots extraits par le vectoriseur et les mots filtrés de la matrice `M_test_quest_words_filtered_nmf`. Les noms des colonnes du DataFrame sont obtenus via `vectorizer.get_feature_names_out()` et les index sont alignés avec ceux de `test_df`. Le résultat est le DataFrame `df_keywords_test_nmf`, qui contient les mots clés pertinents pour chaque document de test :

In [ ]:
words = vectorizer.get_feature_names_out()
df_keywords_test_nmf = pd.DataFrame(M_test_quest_words_filtered_nmf, columns=words, index=test_df.index)

In [ ]:
df_keywords_test_nmf

On va utiliser ce DataFrame `df_keywords_test_nmf` pour créer une colonne `predicted_nmf` dans le DataFrame `test_df` :

In [ ]:
test_df.head()

In [ ]:
test_df['predicted_nmf'] = df_keywords_test_nmf.apply(lambda row: row.nlargest(len(test_df.split_tags)).index.tolist(), axis=1)

In [ ]:
test_df.head()

### Approche semi-supervisée

Pour évaluer la similarité entre les topics des questions de test et ceux des questions d'entraînement, nous utilisons la mesure de similarité cosinus. Cette méthode compare les vecteurs de topics en calculant le cosinus de l'angle entre eux, ce qui permet de quantifier la ressemblance des distributions de topics entre les deux ensembles de questions. Nous appliquons cette méthode aux matrices de topics générées par les modèles LDA et NMF pour obtenir les matrices de similarité correspondantes :

In [ ]:
similarity_matrix_lda = func.calculate_similarity_matrix(M_test_quest_topics_lda, M_train_quest_topics_lda)  # Matrice de similarité pour LDA
similarity_matrix_lda.shape

In [ ]:
similarity_matrix_nmf = func.calculate_similarity_matrix(M_test_quest_topics_nmf, M_train_quest_topics_nmf)  # Matrice de similarité pour NMF
similarity_matrix_nmf.shape

#### Création de la liste du vocabulaire des mots les plus fréquents de corpus `df['split_tags'] `:

In [ ]:
# Convertir les chaînes de caractères en listes pour la colonne 'split_tags'
# Cette étape est nécessaire si 'split_tags' contient des listes de tags encodées en chaînes de caractères
df['split_tags'] = df['split_tags'].apply(ast.literal_eval)

In [ ]:
# Combiner tous les tags en une seule liste de corpus
corpus_tags = [tag for sublist in df['split_tags'] for tag in sublist]

# Afficher la fréquence de chaque tag dans le corpus
value_counts_tags = pd.Series(corpus_tags).value_counts()
print("Fréquence de chaque tag dans le corpus :\n\n",value_counts_tags)

À partir des résultats de `value_counts` obtenus pour le corpus de tags `df['split_tags']`, je sélectionnerai les 200 mots les plus fréquents. Ces mots seront ensuite intégrés dans la liste `vocabulary_tags` :

In [ ]:
vocabulary_tags = list(value_counts_tags.head(200).index)
print(vocabulary_tags)

#### Création des vectoriseurs pour les tags:

In [ ]:
vectorizer_tags = TfidfVectorizer(vocabulary=vocabulary_tags)  # Vectoriseur pour les tags
# vectorizer_tags = TfidfVectorizer(max_features=5000)
X_tags = vectorizer_tags.fit_transform(train_df['split_tags'])  # Transformation des tags d'entraînement
tags_train = X_tags.toarray()  # Conversion en array numpy
tags_train.shape

In [ ]:
with open('Model/unsupervised/vectorizer_tags.pkl', 'wb') as file:
    pickle.dump(vectorizer_tags, file)

In [ ]:
np.savez('Model/unsupervised/tags_train.npz', tags_train=tags_train)

#### Extraire les Mots clés semi-supervisés pour LDA et NMF:

On prend la matrice des topics des questions de test et la matrice de similarité LDA, puis on effectue **une multiplication matricielle** pour produire une matrice des mots clés semi-supervisés. Le résultat, assigné à `keywords_test_semi_supervised_lda`, représente les mots clés filtrés en fonction de leur pertinence pour chaque topic identifié :

In [ ]:
keywords_test_semi_supervised_lda = np.dot(similarity_matrix_lda, tags_train)  # Mots clés semi-supervisés pour LDA
keywords_test_semi_supervised_lda.shape

On prend la matrice des topics des questions de test et la matrice de similarité NMF, puis on effectue **une multiplication matricielle** pour produire une matrice des mots clés semi-supervisés. Le résultat, assigné à `keywords_test_semi_supervised_nmf`, représente les mots clés filtrés en fonction de leur pertinence pour chaque topic identifié :

In [ ]:
keywords_test_semi_supervised_nmf = np.dot(similarity_matrix_nmf, tags_train)  # Mots clés semi-supervisés pour NMF
keywords_test_semi_supervised_nmf.shape

#### Création de DataFrames avec les mots clés proposés par les modèles LDA et NMF en semi-supervisé:

Ensuite, je construis les DataFrames `df_keywords_test_semi_supervised_lda` et `df_keywords_test_semi_supervised_nmf` en utilisant les noms des tags extraits par le vectoriseur et les mots filtrés des matrices `keywords_test_semi_supervised_lda` et `keywords_test_semi_supervised_nmf`. Les noms des colonnes des DataFrames sont obtenus via `vectorizer_tags.get_feature_names_out()` et les index sont alignés avec ceux de `test_df`. Le résultat est deux DataFrames, `df_keywords_test_semi_supervised_lda` et `df_keywords_test_semi_supervised_nmf`, qui contiennent les mots clés pertinents pour chaque document de test :

In [ ]:
tags = vectorizer_tags.get_feature_names_out()
df_keywords_test_semi_supervised_lda = pd.DataFrame(keywords_test_semi_supervised_lda, columns=tags, index=test_df.index)
df_keywords_test_semi_supervised_nmf = pd.DataFrame(keywords_test_semi_supervised_nmf, columns=tags, index=test_df.index)

In [ ]:
df_keywords_test_semi_supervised_lda

In [ ]:
df_keywords_test_semi_supervised_nmf

##### Création de la colonne `predicted_lda_semi_supervised` dans le DataFrame `test_df` :

On va utiliser ce DataFrame `df_keywords_test_semi_supervised_lda` pour créer une colonne `predicted_lda_semi_supervised` dans le DataFrame `test_df` :

In [ ]:
test_df.head()

In [ ]:
test_df['predicted_lda_semi_supervised'] = df_keywords_test_semi_supervised_lda.apply(lambda row: row.nlargest(len(test_df.split_tags.iloc[0])).index.tolist(), axis=1)

In [ ]:
test_df.head()

##### Création de la colonne `predicted_nmf_semi_supervised` dans le DataFrame `test_df` :

On va utiliser ce DataFrame `df_keywords_test_semi_supervised_nmf` pour créer une colonne `predicted_nmf_semi_supervised` dans le DataFrame `test_df` :

In [ ]:
test_df['predicted_nmf_semi_supervised'] = df_keywords_test_semi_supervised_nmf.apply(lambda row: row.nlargest(len(test_df.split_tags.iloc[0])).index.tolist(), axis=1)

In [ ]:
test_df.head()

#### Taux de couverture des tags:

In [ ]:
def coverage_rate(df, actual_column, predicted_column):
    """
    Calcule le taux de couverture entre les tags réels et les tags prédits pour chaque document dans un DataFrame.

    Args:
    - df: Le DataFrame contenant les colonnes des tags réels et prédits.
    - actual_column: Le nom de la colonne contenant les tags réels.
    - predicted_column: Le nom de la colonne contenant les tags prédits.

    Returns:
    - float: Le taux de couverture moyen des tags pour tous les documents.
    """
    def coverage_for_row(row):
        actual_tags = set(row[actual_column])
        predicted_tags = set(row[predicted_column])
        if not actual_tags:
            return 0
        matches = len(actual_tags & predicted_tags)
        total = len(actual_tags)
        return matches / total

    coverage_rates = df.apply(coverage_for_row, axis=1)
    return round(coverage_rates.mean(),4)

#### Calcul du taux de couverture par rapport aux tags réels:

#### Taux de couverture pour le modèle LDA:

In [ ]:
average_coverage_lda = func.coverage_rate(test_df, 'split_tags', 'predicted_lda')
print(f"Taux de couverture moyen pour LDA: {average_coverage_lda:.2f}")

#### Taux de couverture pour le modèle NMF:

In [ ]:
average_coverage_nmf = func.coverage_rate(test_df, 'split_tags', 'predicted_nmf')
print(f"Taux de couverture moyen pour NMF: {average_coverage_nmf:.2f}")

#### Taux de couverture pour le modèle lda_semi_supervised:

In [ ]:
average_coverage_lda_semi_supervised = func.coverage_rate(test_df, 'split_tags', 'predicted_lda_semi_supervised')
print(f"Taux de couverture moyen pour LDA semi-supervisé: {average_coverage_lda_semi_supervised:.2f}")

#### Taux de couverture pour le modèle nmf_semi_supervised:

In [ ]:
average_coverage_nmf_semi_supervised = func.coverage_rate(test_df, 'split_tags', 'predicted_nmf_semi_supervised')
print(f"Taux de couverture moyen pour NMF semi-supervisé: {average_coverage_nmf_semi_supervised:.2f}")

#### Visualisation de la distribution des topics

In [ ]:
# Créer la heatmap pour les topics LDA
func.plot_heatmap(M_test_quest_topics_lda, 'Distribution des Topics LDA pour les Questions Test')

In [ ]:
# Créer la heatmap pour les topics NMF
func.plot_heatmap(M_test_quest_topics_nmf, 'Distribution des Topics NMF pour les Questions Test')

#### Test de visu sur quelques questions

In [ ]:
sample_indices = np.random.choice(test_df.index, 5, replace=False)  # Sélection aléatoire de quelques indices

for i in sample_indices:
    print(f"Question: {test_df.loc[i, 'combined_title_body']}")
    print(f"Tags réels: {test_df.loc[i, 'split_tags']}")
    print(f"Mots clés proposés (LDA): {df_keywords_test_lda.loc[i].nlargest(5).index.tolist()}")  # Mots clés proposés par LDA
    print(f"Mots clés proposés (NMF): {df_keywords_test_nmf.loc[i].nlargest(5).index.tolist()}")  # Mots clés proposés par NMF
    print(f"Mots clés semi-supervisés (LDA): {df_keywords_test_semi_supervised_lda.loc[i].nlargest(5).index.tolist()}")  # Mots clés semi-supervisés par LDA
    print(f"Mots clés semi-supervisés (NMF): {df_keywords_test_semi_supervised_nmf.loc[i].nlargest(5).index.tolist()}")  # Mots clés semi-supervisés par NMF
    print("--------------------------------------------------")

### Création d'un code à tester avant de l'implémenter dans la future API :

#### Charger les modèles et les matrices:

In [ ]:
with open('Model/unsupervised/lda_model.pkl', 'rb') as file:
    lda_model = pickle.load(file)
    
with open('Model/unsupervised/nmf_model.pkl', 'rb') as file:
    nmf_model = pickle.load(file)

with open('Model/unsupervised/vectorizer.pkl', 'rb') as file:
    vectorizer= pickle.load(file)
    
with open('Model/unsupervised/vectorizer_tags.pkl', 'rb') as file:
    vectorizer_tags = pickle.load(file)
    

with open('Model/unsupervised/train_topics_lda.pkl', 'rb') as file:
    train_topics_lda = pickle.load(file)
    
with open('Model/unsupervised/train_topics_nmf.pkl', 'rb') as file:
    train_topics_nmf = pickle.load(file)
    

# Charger la matrice des tags depuis le fichier compressé NPZ
file_path_tags = 'Model/unsupervised/tags_train.npz'
with np.load(file_path_tags) as data:
    tags_train = data['tags_train']

#### Tester le code:

#### library :

In [ ]:
# exemple de texte
texte = '''<p>How do I:</p>
<ol>
<li><p>Create a local branch from another branch (via <code>git branch</code> or <code>git checkout -b</code>).</p>
</li>
<li><p>Push the local branch
to the remote repository (i.e. publish), but make it
trackable so that <code>git pull</code> and <code>git push</code> will work.</p>
</li>
</ol>'''



print(texte)

In [ ]:
# Exemple d'utilisation directe
user_input = texte

if user_input:
    # Nettoyage et tokenisation du texte d'entrée
    cleaned_html_input = func.clean_html(user_input)
    cleaned_input = func.process_clean_text(cleaned_html_input)
    
    predicted_keywords_lda, predicted_semi_supervised_keywords_lda, predicted_keywords_nmf, predicted_semi_supervised_keywords_nmf  = func.predict_keywords(cleaned_input, lda_model, nmf_model, vectorizer, train_topics_lda, train_topics_nmf, tags_train, vectorizer_tags)
    
    print("---------------------------------------")
    print("le texte nettoyé:", cleaned_input)
    print("---------------------------------------")
    print('Mots-clés prédits avec LDA model:')
    print(predicted_keywords_lda)
    print('Mots-clés semi-supervisés prédits LDA:')
    print(predicted_semi_supervised_keywords_lda)
    print("---------------------------------------")
    print('Mots-clés prédits avec NMF model:')
    print(predicted_keywords_nmf)
    print('Mots-clés semi-supervisés prédits NMF:')
    print(predicted_semi_supervised_keywords_nmf)
else:
    print("Veuillez entrer du texte pour la prédiction.")
